In [ ]:
from flask import Flask, jsonify
import requests

app = Flask(__name__)

window_size = 10
numbers_window = []

api_endpoints = {
    'primes': 'http://20.244.56.144/test/primes',
    'fibo': 'http://20.244.56.144/test/fibo',
    'even': 'http://20.244.56.144/test/even',
    'random': 'http://20.244.56.144/test/random'
}

api_credentials = {
    "companyName": "GMRIT",
    "clientID": "4a0fb079-6442-49c7-808b-781cea1d1c40",
    "clientSecret": "svNGnCfKeSulIZsN",
    "ownerName": "Surya Chittimoju",
    "ownerEmail": "21341A1232@gmrit.edu.in",
    "rollNo": "21341A1232"
}

def fetch_numbers(endpoint):
    try:
        auth_token = requests.post('http://20.244.56.144/test/auth',json=api_credentials)
        headers = {
            "Authorization": "Bearer "+auth_token.json().get("access_token","")
        }
        response = requests.get(endpoint, headers = headers, timeout=0.5) 
        return response.json().get("numbers", [])
    except requests.Timeout:
        return None  # Handle timeout error

@app.route('/numbers/<numberid>', methods=['GET'])
def get_numbers(numberid):
    global numbers_window
    
    endpoint = api_endpoints.get(numberid)
    if not endpoint:
        return jsonify({"error": "Invalid number ID"}), 400
    
    numbers = fetch_numbers(endpoint)
    if numbers is None:
        return jsonify({"error": "Request timeout. Try again later."}), 504
    
    windowPrevState = numbers_window.copy()
    
    numbers_window.extend(num for num in numbers if num not in numbers_window)
    if len(numbers_window) > window_size:
        numbers_window = numbers_window[-window_size:] 
    avg = "{:.2f}".format(sum(numbers_window) / len(numbers_window)) if numbers_window else "0.00"
    
    response = {
        "windowPrevState": windowPrevState,
        "windowCurrState": numbers_window,
        "numbers": numbers,
        "avg": avg
    }
    
    return jsonify(response)

if __name__ == "__main__":
    app.run(port=9876)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:9876
Press CTRL+C to quit
127.0.0.1 - - [15/Aug/2024 13:02:22] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2024 13:02:32] "GET /numbers/primes HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2024 13:03:02] "GET /numbers/fibo HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2024 13:03:22] "GET /numbers/rndom HTTP/1.1" 400 -
[2024-08-15 13:03:32,259] ERROR in app: Exception on /numbers/random [GET]
Traceback (most recent call last):
  File "C:\Users\SURYA\anaconda3\Lib\site-packages\requests\models.py", line 971, in json
    return complexjson.loads(self.text, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SURYA\anaconda3\Lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SURYA\anaconda3\Lib\json\decoder.py", line 340, in decode
    raise JSONDecodeError("Extra data", s, end)
json.decoder.JSONDecodeError: Extra data: line 1 column 5 (char 4)

During handling of the

127.0.0.1 - - [15/Aug/2024 13:04:41] "GET /numbers/random HTTP/1.1" 500 -
[2024-08-15 13:04:42,520] ERROR in app: Exception on /numbers/random [GET]
Traceback (most recent call last):
  File "C:\Users\SURYA\anaconda3\Lib\site-packages\requests\models.py", line 971, in json
    return complexjson.loads(self.text, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SURYA\anaconda3\Lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SURYA\anaconda3\Lib\json\decoder.py", line 340, in decode
    raise JSONDecodeError("Extra data", s, end)
json.decoder.JSONDecodeError: Extra data: line 1 column 5 (char 4)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SURYA\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  F